In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Define the path to your training data
train_data_dir = 'train'

# Define the image size
img_width, img_height = 224, 224

In [34]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Define the path to your training data
train_data_dir = 'train2'

# Define the image size
img_width, img_height = 224, 224

# Function to preprocess the images
def preprocess_images(img):
    img = img.astype('float32')
    img /= 255.0
    return img

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    preprocessing_function=preprocess_images
)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1","/gpu:2","/gpu:3","/gpu:4"])

# Создание модели внутри MirroredStrategy
with mirrored_strategy.scope():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
# Add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
# Add an output layer with the number of classes
    num_classes = len(train_generator.class_indices)
    predictions = Dense(num_classes, activation='softmax')(x)
# Create the model
    model = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(train_generator, epochs=20, verbose=1)



Found 369 images belonging to 371 classes.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhos

Epoch 1/20
INFO:tensorflow:batch_all_reduce: 85 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 85 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 85 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 85 all-reduces with algorithm = nccl, num_packs = 1


12/12 [==============================] - 66s 529ms/step - loss: 6.8220 - accuracy: 0.0000e+00
Epoch 2/20
12/12 [==============================] - 8s 653ms/step - loss: 5.8170 - accuracy: 0.0136
Epoch 3/20
12/12 [==============================] - 8s 629ms/step - loss: 5.3385 - accuracy: 0.0840
Epoch 4/20
12/12 [==============================] - 8s 628ms/step - loss: 4.5220 - accuracy: 0.1491
Epoch 5/20
12/12 [==============================] - 8s 614ms/step - loss: 3.5623 - accuracy: 0.3144
Epoch 6/20
12/12 [==============================] - 8s 624ms/step - loss: 2.6118 - accuracy: 0.5610
Epoch 7/20
12/12 [==============================] - 8s 618ms/step - loss: 1.7564 - accuracy: 0.7073
Epoch 8/20
12/12 [==============================] - 8s 625ms/step - loss: 1.0944 - accuracy: 0.8374
Epoch 9/20
12/12 [==============================] - 8s 626ms/step - loss: 0.7045 - accuracy: 0.8753
Epoch 10/20
12/12 [==============================] - 8s 624ms/step - loss: 0.4447 - accuracy: 0.9621
Epoch

InvalidArgumentError: Graph execution error:

Detected at node 'replica_4/categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/home/user/miniconda3/envs/tf/lib/python3.9/threading.py", line 937, in _bootstrap
      self._bootstrap_inner()
    File "/home/user/miniconda3/envs/tf/lib/python3.9/threading.py", line 980, in _bootstrap_inner
      self.run()
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1824, in run_step
      outputs = model.test_step(data)
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1790, in test_step
      self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1109, in compute_loss
      return self.compiled_loss(
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 265

In [42]:
# Сохранение модели
model.save('image_similarity_model.h5')

In [43]:
loaded_model = keras.models.load_model('image_similarity_model.h5')
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [40]:
#!pip install pydot

In [41]:
from keras.utils.vis_utils import plot_model
plot_model(loaded_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [44]:
test_data_dir = 'test'
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical'
)

loss, accuracy = model.evaluate(test_generator)
print('Loss:', loss)
print('Accuracy:', accuracy)


Found 10 images belonging to 10 classes.


InvalidArgumentError: Graph execution error:

Detected at node 'replica_1/categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/home/user/miniconda3/envs/tf/lib/python3.9/threading.py", line 937, in _bootstrap
      self._bootstrap_inner()
    File "/home/user/miniconda3/envs/tf/lib/python3.9/threading.py", line 980, in _bootstrap_inner
      self.run()
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1824, in run_step
      outputs = model.test_step(data)
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1790, in test_step
      self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1109, in compute_loss
      return self.compiled_loss(
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 265

In [3]:
#!pip install opencv-python
#!python --version

In [4]:
import keras
from keras.preprocessing.image import ImageDataGenerator

# Загрузка модели
model = keras.models.load_model('image_similarity_model.h5')
# Define the image size
img_width, img_height = 224, 224

# Function to preprocess the images
def preprocess_images(img):
    img = img.astype('float32')
    img /= 255.0
    return img
# Создание генератора для тестовых данных
test_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    preprocessing_function=preprocess_images
)
test_data_dir = 'train'
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical'
)

# Оценка модели на тестовых данных
loss, accuracy = model.evaluate(test_generator)
print('Loss:', loss)
print('Accuracy:', accuracy)


Found 369 images belonging to 371 classes.
369/369 [==============================] - 7s 19ms/step - loss: 17.2858 - accuracy: 0.0054
Loss: 17.28577423095703
Accuracy: 0.00542005430907011
